# A - 3 - Cleaning and Normalization

**Process aim:** cleaning the dataset and reduce it to data used for machine learning.

**Input:** a CSV containing metadata and full text

**Subprocesses:**
* import: importing the dataset
* dataset reduction: removing documents (rows) that are missing or have an invalid value
* normalization: normalizing so that all values in certain fields have the same format
* enrichement: adding some fields based on others
* cleaning: removing special characters such as new lines, etc.
* subsets: create various subsets from the initial dataset.
* saving the output
**Output:** CSV files

In [6]:
import numpy as np
import pandas as pd
import re

## Import the dataset
We keep only the columns that we will use as/for:
* identificator: record_id
* labels: columns containing the subjects of the documents (geo, topics) that we want to assign automatically in the future
* features: columns containing information we can use to infer the subjects (title, text)
* grouping: columns to be used to group subset of documents (symbol, body, date)

In [9]:
# Create the dataframe
columns = ['record_id','body', 'symbol', 'title', 'date', 'topics', 'geo','text']
dataset = pd.read_csv('../data/A_input/doc_2000_2017_txt.csv', dtype='str', usecols=columns, index_col='record_id')

In [10]:
dataset.columns

Index(['body', 'date', 'geo', 'symbol', 'title', 'topics', 'text'], dtype='object')

In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151804 entries, 455823 to 247308
Data columns (total 7 columns):
body      151802 non-null object
date      151795 non-null object
geo       63147 non-null object
symbol    151802 non-null object
title     151804 non-null object
topics    132022 non-null object
text      128144 non-null object
dtypes: object(7)
memory usage: 9.3+ MB


Using dataset.info() we can see for each field how many are non-null values. For instance, how many records have subject-topics.

## Dataset reduction
Dataset reduction is the process of removing entris (rows) because some fields don't have a valid value. In this case, want to keep only the documents that have a valid value in fields date, symbol, title, and text.

In [13]:
def reduce_dataset(dataset,filter_fields):
    '''
    Takes a list of fields to filter. 
    For each field, remove the rows that don't have a valid value. 
    Return the reduce dataset
    '''
    for field in filter_fields:
        dataset = dataset[dataset[field].notnull()]
    return dataset

In [14]:
# Create a list with the fields we want to filter:
filter_fields = ['date', 'symbol', 'title', 'text']
dataset = reduce_dataset(dataset,filter_fields)

In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128144 entries, 455823 to 273481
Data columns (total 7 columns):
body      128144 non-null object
date      128144 non-null object
geo       54721 non-null object
symbol    128144 non-null object
title     128144 non-null object
topics    112489 non-null object
text      128144 non-null object
dtypes: object(7)
memory usage: 7.8+ MB


## Normalization
### Year
To be able to sort and group by date, we need to ensure that the date is normalized and that all rows have a value. We will use only the year instead of the full date.

In [16]:
dataset.date.unique()

array(['20011206', '20101004', '20041028', ..., '20000101', '20000213',
       '20000316'], dtype=object)

In [17]:
dataset['date'] = (dataset.date
                   .str.extract('(\d{4})',expand=False) #  extract the first 4 digits corresponding to the year
                  )

In [18]:
dataset.date.unique()

array(['2001', '2010', '2004', '2003', '2000', '2014', '2006', '2005',
       '2016', '2015', '2012', '2011', '2009', '2008', '2002', '2013',
       '2007', '2017', nan, '1999'], dtype=object)

In [19]:
# We still find some non-valid values: nan and 1999 (our dataset is supose to cover 2000->)
dataset = dataset[dataset['date'].notnull()]
dataset = dataset[dataset['date'] != '1999']

In [20]:
#  Rename column 'date' to year
dataset = dataset.rename(columns={'date':'year'})

In [21]:
# Check our dataset to ensure the date field has a value for each record
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128142 entries, 455823 to 273481
Data columns (total 7 columns):
body      128142 non-null object
year      128142 non-null object
geo       54720 non-null object
symbol    128142 non-null object
title     128142 non-null object
topics    112487 non-null object
text      128142 non-null object
dtypes: object(7)
memory usage: 7.8+ MB


### Body
Annother usefull grouping field is the body. However, there are many different values. We are going to normalize the field so that we keep only the indication of the main UN organ.

In [22]:
def normalize_body(line):
    '''
    Clean multiple value cells to return multiple values to return each line:
    - fields with multiple values: value one||value two
    - fields with a single value: value one
    '''
    if isinstance(line, str):
        line = re.sub("(\*+)|\'|\[|\]|\s+","",line)
        line = line.split(',')
        line = [body.split("/")[0] for body in line]
        line = [body for body in line if body in ['A','S','E']]
        line = filter(None, line)
        line = list(set(line))
        line = re.sub("(\*+)|\'|\[|\]|\s+","",str(line))
        return line
    else:
        return line

In [23]:
dataset['main_body'] = dataset['body'].apply(normalize_body)

In [24]:
dataset.main_body.unique()

array(['A,S', 'E', 'A', 'S', 'A,E', ''], dtype=object)

In [25]:
# filter out the empty values
dataset = dataset[dataset['main_body'] != '']

In [26]:
dataset.main_body.unique()

array(['A,S', 'E', 'A', 'S', 'A,E'], dtype=object)

In [27]:
# drop the body field now that we have a main_body columns
dataset = dataset.drop('body', axis=1)

In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128141 entries, 455823 to 273481
Data columns (total 7 columns):
year         128141 non-null object
geo          54719 non-null object
symbol       128141 non-null object
title        128141 non-null object
topics       112486 non-null object
text         128141 non-null object
main_body    128141 non-null object
dtypes: object(7)
memory usage: 7.8+ MB


### Multiple values

Some columns have multiple values, we want to normalize these columns as follow:
* fields with multiple values: value one||value two
* fields with a single value: value one

In [30]:
def normalize_multiple(line):
    '''
    Clean multiple value cells to return multiple values to return each line:
    - fields with multiple values: value one||value two
    - fields with a single value: value one
    '''
    if isinstance(line, str):
        if line.startswith('['):
            line = re.sub("\[|\]","",line)
            line = line.strip("'")
            line = line.strip('"')
            line = line.strip(" ")
            line = re.sub("('|\"),\s?('|\")","||",line)
            return line
        else:
            line = line.strip("'")
            line = line.strip('"')
            line = line.strip(" ")
            line = re.sub("\*","",line)
            return line
    else:
        return line

In [31]:
dataset['geo'] = dataset['geo'].apply(normalize_multiple)
dataset['topics'] = dataset['topics'].apply(normalize_multiple)
dataset['symbol'] = dataset['symbol'].apply(normalize_multiple)

In [33]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128141 entries, 455823 to 273481
Data columns (total 7 columns):
year         128141 non-null object
geo          54719 non-null object
symbol       128141 non-null object
title        128141 non-null object
topics       112486 non-null object
text         128141 non-null object
main_body    128141 non-null object
dtypes: object(7)
memory usage: 7.8+ MB


## Add new columns
### Concatenate existing columns
Subjects are stored in fields 'topics' and 'geo' (geographical terms). By concatenating both columns, we can create a new field that contains all the label we intend to predict.

In [34]:
def concatenate_columns(dataset,new_column, column_A, column_B):
    '''
    Takes a dataset, the name of a new column, and two columns to concatenate
    Create the new column, by concatenating columns A and B
    Remove invalid strings and values
    Return the dataset
    '''
    dataset[new_column] = (dataset
                           .apply(lambda x:'%s||%s' % (x[column_A],x[column_B]),axis=1)
                           .apply(lambda x: x.replace('nan||','')) # clean nan strings
                           .apply(lambda x: x.replace('||nan','')) # clean nan strings
                          )
    dataset[new_column] = dataset[new_column].replace('nan',np.nan)
    return dataset

In [35]:
dataset = concatenate_columns(dataset, 'topics_geo', 'topics', 'geo')

We reduce the dataset so that all documents have at least one label

In [36]:
dataset = reduce_dataset(dataset,['topics_geo'])

In [37]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113483 entries, 455823 to 273481
Data columns (total 8 columns):
year          113483 non-null object
geo           54719 non-null object
symbol        113483 non-null object
title         113483 non-null object
topics        112486 non-null object
text          113483 non-null object
main_body     113483 non-null object
topics_geo    113483 non-null object
dtypes: object(8)
memory usage: 7.8+ MB


### Counting values in columns

In [38]:
def count_values(values,separator):
    if isinstance(values,str):
        return len(values.split(separator))
    else:
        return 0

def add_count_fields(dataset, fields):
    for field in fields:
        dataset[field + '_count'] = (dataset[field].apply(lambda x: count_values(x,'||')))
    return dataset

In [39]:
# Create a list of fields
label_fields = ['geo', 'topics', 'topics_geo']
# Add a column that count the number of subjects assigned to each document
dataset = add_count_fields(dataset, label_fields)

In [40]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113483 entries, 455823 to 273481
Data columns (total 11 columns):
year                113483 non-null object
geo                 54719 non-null object
symbol              113483 non-null object
title               113483 non-null object
topics              112486 non-null object
text                113483 non-null object
main_body           113483 non-null object
topics_geo          113483 non-null object
geo_count           113483 non-null int64
topics_count        113483 non-null int64
topics_geo_count    113483 non-null int64
dtypes: int64(3), object(8)
memory usage: 10.4+ MB


## Cleaning
### Text

We also need to clean the text that contains many special characters indicating a new line, etc.

In [41]:
dataset['text'].head(10)

record_id
455823    A/56/682–S/2001/1159\n\nUnited Nations\n\nGene...
694579    E/2010/SR.46\n\nUnited Nations\n\nEconomic and...
550037    E/2004/SR.47\n\nUnited Nations\n\nEconomic and...
524202    PROVISIONAL\nE/2003/SR.49\n28 November 2003\nO...
420454    A/55/257–S/2000/766\n\nUnited Nations\n\nGener...
536496    A/59/PV.65\n\nUnited Nations\n\nGeneral Assemb...
784297    A/68/966–S/2014/573\n\nUnited Nations\n\nGener...
568031    S/2006/113\n\nUnited Nations\n\nSecurity Counc...
502156    A/57/759–S/2003/332\n\nUnited Nations\n\nGener...
563509    S/RES/1645 (2005)\n\nUnited Nations\n\nSecurit...
Name: text, dtype: object

In [42]:
# Replace special characters such as new lines with a space.
dataset['text'] = dataset['text'].str.replace(r'[\n\t\v]+', ' ')

In [43]:
dataset['text'].head(10)

record_id
455823    A/56/682–S/2001/1159 United Nations General As...
694579    E/2010/SR.46 United Nations Economic and Socia...
550037    E/2004/SR.47 United Nations Economic and Socia...
524202    PROVISIONAL E/2003/SR.49 28 November 2003 Orig...
420454    A/55/257–S/2000/766 United Nations General Ass...
536496    A/59/PV.65 United Nations General Assembly Off...
784297    A/68/966–S/2014/573 United Nations General Ass...
568031    S/2006/113 United Nations Security Council Dis...
502156    A/57/759–S/2003/332 United Nations General Ass...
563509    S/RES/1645 (2005) United Nations Security Coun...
Name: text, dtype: object

In [44]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113483 entries, 455823 to 273481
Data columns (total 11 columns):
year                113483 non-null object
geo                 54719 non-null object
symbol              113483 non-null object
title               113483 non-null object
topics              112486 non-null object
text                113483 non-null object
main_body           113483 non-null object
topics_geo          113483 non-null object
geo_count           113483 non-null int64
topics_count        113483 non-null int64
topics_geo_count    113483 non-null int64
dtypes: int64(3), object(8)
memory usage: 10.4+ MB


## Subsets
Using the field main_body, it is possible to create 3 distincts subsets for documents of the Genearal Assembly(A), the Security Council (S), or the Economic and Social Council (E).

In [ ]:
security_council = dataset[dataset['main_body'].str.contains('S')]
GA_all = dataset[dataset['main_body'].str.contains('A')]
ECOSOC_Council = dataset[dataset['main_body'].str.contains('E')]

We can even create more subsets using the symbol field.

In [ ]:
First_Committee = dataset[dataset['symbol'].str.contains('A/C.1')]
Second_Committee = dataset[dataset['symbol'].str.contains('A/C.2')]
Third_Committee = dataset[dataset['symbol'].str.contains('A/C.3')]
Fourth_Committee = dataset[dataset['symbol'].str.contains('A/C.4')]
Fifth_Committee = dataset[dataset['symbol'].str.contains('A/C.5')]
Sixth_Committee = dataset[dataset['symbol'].str.contains('A/C.6')]
HR_Council = dataset[dataset['symbol'].str.contains('A/HRC')]
GA_Pleanary = GA_all[GA_all['symbol'].str.match('A\/([1-9]|DEC|RES|BUR|PV|INF|SR|ES-|S-)')]

## Save
We can now save our outputs in CSV

In [ ]:
dataset.to_csv('../data/B_engineering/doc_2000_2017.csv')
security_council.to_csv('../data/B_engineering/sc_2000_2017.csv')
GA_all.to_csv('../data/B_engineering/ga_2000_2017.csv')
ECOSOC_Council.to_csv('../data/B_engineering/ecosoc_2000_2017.csv')
First_Committee.to_csv('../data/B_engineering/first_2000_2017.csv')
Second_Committee.to_csv('../data/B_engineering/second_2000_2017.csv')
Third_Committee.to_csv('../data/B_engineering/third_2000_2017.csv')
Fourth_Committee.to_csv('../data/B_engineering/fourth_2000_2017.csv')
Fifth_Committee.to_csv('../data/B_engineering/fifth_2000_2017.csv')
Sixth_Committee.to_csv('../data/B_engineering/sixth_2000_2017.csv')
HR_Council.to_csv('../data/B_engineering/hrc_2000_2017.csv')
GA_Pleanary.to_csv('../data/B_engineering/ga_plen_2000_2017.csv')